In [7]:
import pandas as pd

K = 25

In [9]:
def parser(file_name, sheet_name):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    matches = []
    player_names = []
    scores = []

    for i, row in df.iterrows():
        player = row[0]  # Column A (Player)
        score = row[1]   # Column B (Score)

        if pd.isna(player) and pd.isna(score):
            if len(player_names) in [2, 3, 4, 5]:
                matches.append({'player_names': player_names, 'scores': scores})
            
            player_names = []
            scores = []
        else:
            player_names.append(player)
            scores.append(score)

    # Ensure any leftover data is added as the last match
    if len(player_names) in [2, 3, 4, 5]:
        matches.append({'player_names': player_names, 'scores': scores})

    print(f"{sheet_name} total number of matches is: {len(matches)}")
    return matches

In [10]:
# returns expected score for player A with rating Ra
# expeced score is win probability + 0.5 * draw probability
def calculate_expected_score(Ra, Rb):
    Ea = 1 / (1 + pow(10, (Rb-Ra)/400))
    return Ea

In [11]:
# (player_id : name)
players = pd.read_excel("Data.xlsx", sheet_name="Players", header=None)
players.columns = players.iloc[0]
players = players.drop(0).reset_index(drop=True)

# Convert to dictionary (ID as key, Player as value)
id_to_player_dict = pd.Series(players['Player'].values, index=players['ID']).to_dict()

player_to_id_dict = {v: k for k, v in id_to_player_dict.items()}
    
# (player_id : rating)
ratings_dict = {i: 1500 for i in range(1, len(id_to_player_dict)+1)}

In [12]:
# we'll need a dictionary with current ratings, something like a database for 100+ players
# we also need to have a function that calculates updated ratings based on a match

# gets a list of players (in fact player ids and corresponding points in a match and then
# updates elo for all of them

def update_elo(players, points):
    expected_scores = [0] * len(players)
    actual_scores = [0] * len(players)
    for idx_A, player_A in enumerate(players):
        for idx_B, player_B in enumerate(players):
            if idx_A != idx_B:
                expected_scores[idx_A] += calculate_expected_score(ratings_dict[player_A], ratings_dict[player_B])
                actual_scores[idx_A] += 1 * int(points[idx_A] > points[idx_B]) + 0.5 * int(points[idx_A] == points[idx_B])
    
    for idx_A, player in enumerate(players):
        ratings_dict[player] += K * (actual_scores[idx_A] - expected_scores[idx_A])
    return expected_scores, actual_scores

tournament_sequence = [
    "Vuelta_1",
    "Bez_Povoda",
    "Poligon",
    "USA",
    "Nevermore_1",
    "Nevermore_2",
    "Vuelta_2"
]

for tournament in tournament_sequence:
    matches = parser("Data.xlsx", tournament)
    for match in matches:
        players = [player_to_id_dict[player] for player in match['player_names']]
        scores = match['scores']
        update_elo(players, scores)
    
sorted_rating = [[id_to_player_dict[id], ratings_dict[id]] for id in id_to_player_dict]
sorted_rating = sorted(sorted_rating, key = lambda x: -x[1])
for player, elo in sorted_rating:
    print(f"{player}: {elo:.0f}")

Vuelta_1 total number of matches is: 48
Bez_Povoda total number of matches is: 42
Poligon total number of matches is: 15
USA total number of matches is: 55
Nevermore_1 total number of matches is: 14
Nevermore_2 total number of matches is: 14
Vuelta_2 total number of matches is: 50
Сергей Лобачёв: 1817
Юрий Корчемкин: 1797
Вадим Барановский: 1772
Андрей Романчев: 1761
Шакир Мамедзаде: 1696
Станислав Алимпов: 1685
Николай Арчак: 1675
Евгений Затуловский: 1658
Владимир Грамагин: 1629
Григорий Смыслов: 1608
Илья Миронов: 1602
Иван Суманеев : 1600
Алексей Чернов: 1594
Григорий Львович: 1591
Ирина Соколова: 1581
Евгений Новиков: 1549
Леонид Койфман: 1545
Юлия Жоголева: 1541
Арнур Нигметов: 1540
Жора Яковлев: 1536
Константин Бриф: 1528
Слава Бельков: 1528
Петр Мартынов: 1528
Роман Козелов: 1521
Сергей Меньшиков: 1516
Виктор Исаев: 1515
Макс Галкин: 1512
Ефим Подвойский: 1508
Тимур Мухаматулин: 1507
Евгения Никушина: 1507
Руслан Лепшоков: 1505
Марианна Выдревич: 1504
Григорий Гольберг: 1500
Ил